# Decision Tree Modeling from Scratch with Optimization

# 0. Import Libraries/Packages, Load Data

In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import time
import itertools
import math
import os
import threading
import concurrent.futures
import numba as nb
import multiprocessing as mp

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score
from collections import Counter
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import wait
from concurrent.futures import as_completed
from joblib import Parallel, delayed
from numba import njit

/Users/alexherron/opt/miniconda3/lib/python3.8/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
# Load in data
df = pd.read_csv('cleaned_bullying.csv')

# Drop columns
df = df.drop(columns=['Unnamed: 0',
                      'Bullied_on_school_property_in_past_12_months',
                      'Bullied_not_on_school_property_in_past_12_months',
                      'Cyber_bullied_in_past_12_months'])

# 1. Decision Tree Model from Scratch

In [3]:
# Brute force approach:
# Generate all possible binary splits of the data
# using itertools.combinations
def generate_possible_splits(data):
    
    # Get all features (aside from last column)
    features = list(data.columns)[:-1]
    
    # Initialize splits
    splits = []
    
    # Loop through each feature
    for feature in features:
        
        # Get unique values for current feature
        values = data[feature].unique()
        
        # Loop through different split sizes
        for i in range(1, len(values)):
            
            # Generate all combinations using itertools, 
            # add to splits list
            for split in itertools.combinations(values, i):
                splits.append((feature, split))
                
    return splits

# Evaluate each split based on entropy / information gain,
# where information gain is the change in entropy
def evaluate_split(split, data):
    
    # Unpack features and values from split
    feature, values = split
    
    # Calculate number of instances
    total = len(data)
    
    # Split data into left/right subsets
    # based on feature values
    left = data[data[feature].isin(values)]
    right = data[~data[feature].isin(values)]
    
    # Calculate proportion of instances in 
    # left/right subsets
    left_proportion = len(left) / total
    right_proportion = len(right) / total
    
    # Calculate entropy before/after split and info gain,
    # which is the difference of the two
    entropy_before = calculate_entropy(data)
    entropy_after = (left_proportion * calculate_entropy(left) +
                     right_proportion * calculate_entropy(right))
    information_gain = entropy_before - entropy_after
    
    # Return info gain and left/right subsets
    return information_gain, left, right

# Calculate entropy of data
# for evaluate_split() function
def calculate_entropy(data):
    
    # Calculate number of instances
    total = len(data)
    
    # Count occurences for each label in last column
    counts = data.iloc[:, -1].value_counts()
    
    # Initialize entropy
    entropy = 0
    
    # Calculate entropy for current class label
    # entropy = -sum(p * log_2(p))
    for count in counts:
        proportion = count / total
        entropy -= proportion * math.log2(proportion)
        
    return entropy

# Build decision tree using brute force
def build_decision_tree(data):
    
    # If all instances have the same class, 
    # create leaf node with class label
    if len(data.iloc[:, -1].unique()) == 1:
        return {'class': data.iloc[0, -1]}
    
    # Initalize best split and best gain
    best_split = None
    best_gain = 0
    
    # Loop over all possible splits
    for split in generate_possible_splits(data):
        
        # Evaluate information gain for each split
        gain, left, right = evaluate_split(split, data)
        
        # Update best split if current split has greater
        # information gain
        if gain > best_gain:
            best_gain = gain
            best_split = split
            best_left = left
            best_right = right
            
    # If no split improved information gain,
    # create leaf node with majority class label
    if best_split is None:
        return {'class': data.iloc[:, -1].value_counts().idxmax()}
    else:
        
        # Recursively call build_decision_tree function
        # to build left and right subtrees
        left_subtree = build_decision_tree(best_left)
        right_subtree = build_decision_tree(best_right)
        
        # Create decision node with best split feature / value,
        # including left/right subtrees
        return {'feature': best_split[0],
                'value': best_split[1],
                'left': left_subtree,
                'right': right_subtree}

# Calculate y_pred to compare with y_true
def predict(instance, tree):    
    
    # If current node is a leaf node
    if 'class' in tree:
        # Return predicted class
        return tree['class']
    
    else:
        feature = tree['feature']
        value = tree['value']
        
        # If value of feature satisfies condition
        # add to left subtree
        if instance[feature] in value:
            subtree = tree['left']
            
        # Otherwise, add to right subtree
        else:
            subtree = tree['right']
            
        # Call function recursively until a leaf 
        # node is reached
        return predict(instance, subtree)
    
# Single function to split data, train decision tree model,
# evaluate model, and print evaluation metrics 
def evaluate():
    
    # Load dataset, split into train / test sets
    data = df
    train_data = data.sample(frac=0.8, random_state=1)
    test_data = data.drop(train_data.index)

    # Train decision tree model 
    # (calculate time required to train)
    start = time.time()
    tree = build_decision_tree(train_data)
    end = time.time()
    time_diff = end - start

    # Evaluate decision tree
    y_true = test_data.iloc[:, -1]
    y_pred = test_data.apply(lambda x: predict(x, tree), axis=1)
    accuracy = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(pd.get_dummies(y_true), pd.get_dummies(y_pred))

    # Print results
    print(f'AUC: {auc}')
    print(f'Accuracy: {accuracy}')
    print(f'Time: {time_diff}')
    
    # Make beeping sounds to notify end of evaluation
    beep = lambda x: os.system("echo '\a';sleep 0.5;" * x)
    beep(5)

In [4]:
evaluate()

AUC: 0.5920597098471023
Accuracy: 0.6158166363084396
Time: 289.3000137805939







# 2. Evalute Decision Tree Model Using Line Profiler

~96% of time spent on evaluate_split(split, data)

In [5]:
# Load copy of dataframe
data = df

# Train/test split
train_data = data.sample(frac=0.8, random_state=1)
test_data = data.drop(train_data.index)

In [6]:
%load_ext line_profiler

In [7]:
%lprun -f build_decision_tree tree = build_decision_tree(train_data)

# 3. Generate Random Binary Splits

- Avoid calculating info gain for all possible splits (brute force)
- Instead, choose random subset of features to split on (reducing number of splits that need to be evaluted)
- Implemented by changing 2 functions: 
    - generate_possible_splits
    - evaluate_split

In [8]:
# Main idea: generate random binary splits of the data:
# Randomly sample a subset of the features to split on
# This reduces the number of splits that need to be evaluated,
# but still allows the algorithm to find a diverse range of splits.
def generate_possible_splits(data):
    
    # Get all features (aside from last column)
    # and random sample sizes
    features = list(data.columns)[:-1]
    rand_a, rand_b = 5, 3
    
    # Randomly sample features (without replacement)
    features = list(np.random.choice(features, rand_a, replace=False))

    # Initialize splits
    splits = []
    
    # Loop through each feature
    for feature in features:
        
        # Get unique values for current features
        values = data[feature].unique()
        
        # Randomly sample values without replacement
        values = list(np.random.choice(values, min(rand_b, len(values)), replace=False))
        
        # Loop through different split sizes
        for i in range(1, len(values)):
            
            # Generate all combinations using itertools, 
            # add to splits list
            for split in itertools.combinations(values, i):
                splits.append((feature, split))
                
    return splits

In [9]:
evaluate()

AUC: 0.602041000199365
Accuracy: 0.6317547055251973
Time: 54.09505915641785







# 4. Optimize Decision Trees with Threading

### 4.a. Threading for evaluate_split function

In [10]:
# Using threading (with 4 threads)
def evaluate_split(split, data, num_threads=4):
    
    # Unpack features and values from split
    feature, values = split

    # Calculate number of instances
    total = len(data)
    
    # Split data into left/right subsets
    # based on feature values
    left = data[data[feature].isin(values)]
    right = data[~data[feature].isin(values)]
    
    # If left or right subset is empty,
    # return negative infinity (for no info gain)
    if len(left) == 0 or len(right) == 0:
        return -float('inf'), None, None
    
    # Calculate proportion of instances in 
    # left/right subsets
    left_proportion = len(left) / total
    right_proportion = len(right) / total

    # Use threading to calculate entropy for each subset
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        
        # Submit entropy calculation for data, left/right subsets to executor
        futures = [executor.submit(calculate_entropy, subset) for subset in [data, left, right]]
        
        # Retrieve entropy results
        results = [f.result() for f in futures]
    
    # Calculate entropy before/after split and info gain,
    # which is the difference of the two
    entropy_before= results[0]
    entropy_after = (left_proportion * results[1] +
                     right_proportion * results[2])
    information_gain = entropy_before - entropy_after
    
    # Return info gain and left/right subsets
    return information_gain, left, right

In [11]:
evaluate()

AUC: 0.6007529866425384
Accuracy: 0.6323618700667881
Time: 71.76984810829163







### 4.b Threading for build_decision_tree() function and evaluate_split() function

In [12]:
def build_decision_tree(data):

    # If all instances have the same class, 
    # create leaf node with class label
    if len(data.iloc[:, -1].unique()) == 1:
        return {'class': data.iloc[0, -1]}
    
    # Initalize best split and best gain
    best_split = None
    best_gain = 0
    
    # Lock threading
    split_lock = threading.Lock()
    
    # Initialize best left/right splits
    best_left = None  
    best_right = None 
    
    # Create helper function for evaluating/updating best splits
    def evaluate_and_update_split(split):
        nonlocal best_split, best_gain, best_left, best_right
        gain, left, right = evaluate_split(split, data)
        with split_lock:
            if gain > best_gain:
                best_gain = gain
                best_split = split
                best_left = left
                best_right = right
    
    # Create list for threads
    threads = []
    
    # Loop through each split, evaluate each concurrently
    for split in generate_possible_splits(data):

        # Make thread for each split
        thread = threading.Thread(target=evaluate_and_update_split, args=(split,))

        # Start thread, append to thread list
        thread.start()
        threads.append(thread)
    
    # Join threads when they all complete
    for thread in threads:
        thread.join()
    
    # If no split improved information gain,
    # create leaf node with majority class label
    if best_split is None:
        return {'class': data.iloc[:, -1].value_counts().idxmax()}
    else:
        
        # Recursively call build_decision_tree function
        # to build left and right subtrees
        left_subtree = build_decision_tree(best_left)
        right_subtree = build_decision_tree(best_right)
        
        # Create decision node with best split feature / value,
        # including left/right subtrees
        return {'feature': best_split[0],
                'value': best_split[1],
                'left': left_subtree,
                'right': right_subtree}

In [13]:
evaluate()

AUC: 0.6122680463753891
Accuracy: 0.6414693381906497
Time: 81.38614583015442







# 5.a Optimizing Model by Adding Variables for Number of Features and Number of Values

In [14]:
# Add adjustable variables for number of features 
# and number of values
def generate_possible_splits(data, num_features, num_values):
    
    # Get all features (aside from last column)
    features = list(data.columns)[:-1]
    
    # Randomly sample features (using num_features)
    features = list(np.random.choice(features, num_features, replace=False))

    # Initialize splits
    splits = []
    
    # Loop through each feature
    for feature in features:
        
        # Get unique values for current features
        values = data[feature].unique()
        
        # Randomly sample values (using num_values)
        values = list(np.random.choice(values, min(num_values, len(values)), replace=False))

        # Loop through different split sizes
        for i in range(1, len(values)):
            
            # Generate all combinations using itertools, 
            # add to splits list
            for split in itertools.combinations(values, i):
                splits.append((feature, split))
                
    return splits

# Use evaluate_split function without threading
def evaluate_split(split, data):
    
    # Unpack features and values from split
    feature, values = split
    
    # Calculate number of instances
    total = len(data)
    
    # Splits data into left/right instances
    # based on features values
    left = data[data[feature].isin(values)]
    right = data[~data[feature].isin(values)]
    
    # If left or right subset is empty,
    # return None
    if len(left) == 0 or len(right) == 0:
        return None
    
    # Calculate proportion of instances in
    # left/right subsets
    left_proportion = len(left) / total
    right_proportion = len(right) / total
    
    # Calculate entropy before/after split and info gain,
    # which is the difference of the two
    entropy_before = calculate_entropy(data)
    entropy_after = (left_proportion * calculate_entropy(left) +
                     right_proportion * calculate_entropy(right))
    information_gain = entropy_before - entropy_after

    # If info gain is negative, return None
    if information_gain <= 0:
        return None
    
    # Return info gain and left/right subsets
    return information_gain, left, right, split


# Use build_decision_tree function without threading
def build_decision_tree(data, num_features, num_values):

    # If all instances have the same class, 
    # create leaf node with class label
    if len(data.iloc[:, -1].unique()) == 1:
        return {'class': data.iloc[0, -1]}
    
    # Generate possibel splits with adjustable numbers of features/values
    splits = generate_possible_splits(data, num_features, num_values)
    
    # Evaluate each split, store those with non-zero info gain
    splits_gain = [split_gain for split_gain in (evaluate_split(split, data) for split in splits) if split_gain]
    
    # If no split improves info gain, return leaf node with majority class
    if not splits_gain:
        return {'class': data.iloc[:, -1].value_counts().idxmax()}

    # Find best split (highest info gain)
    best_gain, best_left, best_right, best_split = max(splits_gain, key=lambda x: x[0])

    # Build left/right trees recursively
    left_subtree = build_decision_tree(best_left, num_features, num_values)
    right_subtree = build_decision_tree(best_right, num_features, num_values)

    # Return node with best split information
    return {'feature': best_split[0],
            'value': best_split[1],
            'left': left_subtree,
            'right': right_subtree}
    
# Get rid of beeping, allow for changing number of 
# features and values
def evaluate(num_feat, num_val):

    # Load dataset, split into train / test sets
    data = df
    train_data = data.sample(frac=0.8, random_state=1)
    test_data = data.drop(train_data.index)

    # Train decision tree model 
    # (calculate time required to train)    
    start = time.time()
    tree = build_decision_tree(train_data, num_features=num_feat, num_values=num_val)
    end = time.time()
    time_diff = end - start

    # Evaluate decision tree
    y_true = test_data.iloc[:, -1]
    y_pred = test_data.apply(lambda x: predict(x, tree), axis=1)
    accuracy = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(pd.get_dummies(y_true), pd.get_dummies(y_pred))

    # Print results
    print(f'AUC: {auc}')
    print(f'Accuracy: {accuracy}')
    print(f'Time: {time_diff}')

In [15]:
evaluate(3, 3)

AUC: 0.6068284463324489
Accuracy: 0.6419247115968427
Time: 25.457530736923218


# 5.b Basic Hyperparameter Tuning (Number of Features & Number of Values

- num_features, num_values = 2, 2 appears to have the best balance of predicting power and performance

In [16]:
# Collect permutations of all parameters below 4
num_features_values_list = [1, 1, 2, 2, 3, 3]
permuts = itertools.permutations(num_features_values_list, 2)
param_list = []
for i in permuts:
    if i not in param_list:
        param_list.append(i)
    
# Evaluate each number of features / number of values pair
for param_pair in param_list:
    print(f'Number of features: {param_pair[0]}')
    print(f'Number of values: {param_pair[1]}')
    evaluate(param_pair[0], param_pair[1])
    print('\n')

Number of features: 1
Number of values: 1
AUC: 0.5
Accuracy: 0.5980570734669095
Time: 0.0012850761413574219


Number of features: 1
Number of values: 2
AUC: 0.5725416749735457
Accuracy: 0.6396478445658773
Time: 0.39314794540405273


Number of features: 1
Number of values: 3
AUC: 0.5781940589200547
Accuracy: 0.6435944140862173
Time: 0.5821387767791748


Number of features: 2
Number of values: 1
AUC: 0.5
Accuracy: 0.5980570734669095
Time: 0.0013942718505859375


Number of features: 2
Number of values: 2
AUC: 0.6038546475071696
Accuracy: 0.6510321797207043
Time: 4.513548135757446


Number of features: 2
Number of values: 3
AUC: 0.5991628582820863
Accuracy: 0.6425318761384335
Time: 11.353196144104004


Number of features: 3
Number of values: 1
AUC: 0.5
Accuracy: 0.5980570734669095
Time: 0.00164794921875


Number of features: 3
Number of values: 2
AUC: 0.6040446194426979
Accuracy: 0.6451123254401943
Time: 12.749154090881348


Number of features: 3
Number of values: 3
AUC: 0.6030569954146028